## CSC 177-02 Data Warehousing and Data Mining
### Mini-Project 1: Clustering
### 2016 US presedential election Twitter analysis

#### Group members: Aaron Enberg,

In [1]:
import pandas as pd
import sklearn.feature_extraction.text as sk_text
pd.set_option('display.max_colwidth', -1)

In [2]:
column_names = ['Name', 'screen_Name', 'User_ID', 
                'Followers_Count', 'Friends_Count', 
                'Location', 'Description', 'Created_At', 
                'Status_ID', 'Language', 'Place', 
                'Retweet_Count', 'Favorite_Count', 'Text']
tweets = pd.read_table('data/clinton_trump_tweets.txt', names=column_names, encoding='ISO-8859-1')
tweets.columns = tweets.columns.str.lower()

In [3]:
tweets.shape

(5250980, 14)

In [4]:
tweets.dtypes

name               object
screen_name        object
user_id            int64 
followers_count    int64 
friends_count      int64 
location           object
description        object
created_at         object
status_id          int64 
language           object
place              object
retweet_count      int64 
favorite_count     int64 
text               object
dtype: object

## Preprocessing

In [5]:
tweets.drop(['name', 'user_id', 
            'followers_count', 
            'friends_count', 
            'location', 
            'description', 
            'created_at',
            'status_id', 
            'language', 
            'place', 
            'retweet_count', 
            'favorite_count'], axis=1, inplace=True)

In [6]:
tweets.head(n=10)

,screen_name,text
0,Cebel6,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
1,Cookiemuffen,RT @wikileaks: New poll puts Pirate Party on course to win Iceland's national elections on Saturday. https://t.co/edTqjeJaQ6
2,nolaguy_phd,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
3,marksnark,RT @LOLGOP: ACA needs fixes but know da facts: *70% can get covered in marketplaces for under $75/month *Hikes affect 3% *GOP will uninsu
4,aaronjhoddinott,RT @FastCompany: Alphabet shares soar on better-than-expected earnings as mobile video strategy pays off https://t.co/bokbXngMJt https://t.
5,BigPapiCarnivor,RT @Pattric: That one follower who tweet you everyday and even tho you don't respond they still be @'ing you https://t.co/QBeQyc2PO6
6,davidjwalling,#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv
7,robert2266,"Hacked e-mails show Clinton campaigns fears about Sanders | https://t.co/WMyCHuCDIc The Philippine Star (PhilippineStar) October 28, 2"
8,Brianna__Loves,RT @africaarguments: This year's seen some remarkable new films by African directors. Here are a few of them. https://t.co/9JEY4ubGVE http
9,PTicks,"RT @HalifaxEditor: Some 40 years later, @HfxRegPolice continue to investigate the murder of Florence Ann Keeble. https://t.co/axdvA1HUcu ht"


In [7]:
pattern = r'^RT\s'
 
# matches retweets and removes them
tweets = tweets[tweets.text.str.match(pattern) == False]

In [8]:
tweets.shape

(2416818, 2)

In [9]:
tweets.head(n=10)

,screen_name,text
0,Cebel6,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
2,nolaguy_phd,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
6,davidjwalling,#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv
7,robert2266,"Hacked e-mails show Clinton campaigns fears about Sanders | https://t.co/WMyCHuCDIc The Philippine Star (PhilippineStar) October 28, 2"
10,neddyo,Hulk smash!
12,twentythreeofme,"Well, I'm glad you got a good haircut, America's in the toilet. Is this not a conversation @Haylie_Bre and I woul https://t.co/6dVIIKZcgv"
13,CaptainNormal,@WayneDupreeShow #climatechange is the democrats' attempt to play God
18,matthewcbarnard,OH MY GOD. TEARS. https://t.co/8odjnHZwRK
20,_Poochivicious,"no lie, I been waiting for tony's story https://t.co/S5i4ESNilb"
21,valukind,How a dot-com millionaire became a social media crusader against solar amendment: It was a violation of Googl... https://t.co/ychypPZpKd


In [10]:
# match all hashtags and mentions in a tweet, ignoring possible email addresses
pattern = r'(?<=^|(?<=[^a-zA-Z0-9-\.]))@([A-Za-z_]+[A-Za-z0-9_]+)|(?<=^|(?<=[^a-zA-Z0-9-\.]))#([A-Za-z_]+[A-Za-z0-9_]+)'

""" returns a DataFrame with a MultiIndex:
    First index is our original index. Second index is "match" which is a running
    total of the number of occurences of hashtags and mentions for a particular 
    tweet. """
handles_hashtags = tweets.text.str.extractall(pattern)

In [11]:
handles_hashtags.columns = ['handles', 'hashtags']
handles_hashtags = handles_hashtags.reset_index().set_index('level_0')
del handles_hashtags.index.name
handles_hashtags.drop(['match'], axis=1, inplace=True)
# stack handles and hashtags into one column
handles_hashtags = pd.concat([handles_hashtags.handles, handles_hashtags.hashtags]).dropna().to_frame(name='handles_hashtags')

In [12]:
''' returns users along with all the hashtags/handles they've used 
    (each occurrence of a hashtag/handle will also show up in the list) '''
tweets = tweets.join(handles_hashtags, how='inner')
tweets.drop(['text'], axis=1, inplace=True)
handles_hashtags_all = tweets.reset_index().groupby('screen_name')['handles_hashtags'].apply(list).to_frame()

In [13]:
handles_hashtags_all.head(n=10)

,handles_hashtags
screen_name,
000becky000,[annaafletcher]
001_sandhu,[RannvijaySingha]
001comedian,"[Mahasidi, tim_oballa, HassanAliJoho, Amicos, UhurutoCorruptionTrain, Mahasidi, AIRTEL_KE, DeportBobCollymore, HassanAliJoho, Mahasidi, viviankenya, GainWithXtianDela, Mahasidi, GainWithVerifiedCofek, shipetamathias, GainWithVerifiedCofek, UhurutoCorruptionTrain, MaskiniMilionea, AJENews, IWantAPhoneThatCan, UhurutoCorruptionTrain, GainWithVerifiedCofek, GainWithYahBoiSelfie, IWantAPhoneThatCan, njooro, Mahasidi, Fisi, JailAwaitsYou, GainWithVerifiedCofek, paulkiarie001, ngathokaranja, AJEnglish, SonkoTheDrunk, ngathokaranja, AJEnglish, SafaricomLtd, DeportBobCollymore, IWantAPhoneThatCan]"
001nicola,"[XFactor, lifegoals]"
006mariee,[taeccool]
006what,"[clownshoesbeer, photo, shipyardbrewing, photo, NoLabelBrewCo, photo, TallgrassBeer, photo, rogueales, photo, saintarnold, untappd, untappd, trickorbrew]"
007Hanni,[Jaguars]
007_Charleen,"[OnlyInBOS, DunkinDonuts, kelleratlarge]"
007_Digg,"[SenSanders, ThinkerSingher, BernieSanders, alreadycashedclintoncheck]"


### active users that have used at least 20 distinct hashtags/handles

In [14]:
# returns users along with the distinct hashtags/handles they've used
handles_hashtags_distinct = tweets.reset_index().groupby('screen_name')['handles_hashtags'].unique().to_frame()
# retrieve only those who have used 20 or more distinct handles/hashtags
users_active = handles_hashtags_distinct[handles_hashtags_distinct.handles_hashtags.str.len() > 19]

In [15]:
users_active = users_active.join(handles_hashtags_all, lsuffix='_distinct', rsuffix='_all', how='inner')

In [16]:
users_active.head()

,handles_hashtags_distinct,handles_hashtags_all
screen_name,,
001comedian,"[Mahasidi, tim_oballa, HassanAliJoho, Amicos, UhurutoCorruptionTrain, AIRTEL_KE, DeportBobCollymore, viviankenya, GainWithXtianDela, GainWithVerifiedCofek, shipetamathias, MaskiniMilionea, AJENews, IWantAPhoneThatCan, GainWithYahBoiSelfie, njooro, Fisi, JailAwaitsYou, paulkiarie001, ngathokaranja, AJEnglish, SonkoTheDrunk, SafaricomLtd]","[Mahasidi, tim_oballa, HassanAliJoho, Amicos, UhurutoCorruptionTrain, Mahasidi, AIRTEL_KE, DeportBobCollymore, HassanAliJoho, Mahasidi, viviankenya, GainWithXtianDela, Mahasidi, GainWithVerifiedCofek, shipetamathias, GainWithVerifiedCofek, UhurutoCorruptionTrain, MaskiniMilionea, AJENews, IWantAPhoneThatCan, UhurutoCorruptionTrain, GainWithVerifiedCofek, GainWithYahBoiSelfie, IWantAPhoneThatCan, njooro, Mahasidi, Fisi, JailAwaitsYou, GainWithVerifiedCofek, paulkiarie001, ngathokaranja, AJEnglish, SonkoTheDrunk, ngathokaranja, AJEnglish, SafaricomLtd, DeportBobCollymore, IWantAPhoneThatCan]"
007darB,"[Colossis2, dllynnnnn, amyschumer, twitter, LauraJSilverman, FLOTUS, JaclynGlenn, GMA, YouTube, NRO, SarahKSilverman, Forbes, LeslieMarshall, JtMobleyFla, bawesome84, SouthwestAir, KABCRadio, KennedyNation, HillaryClinton, FullFrontalSamB, greggutfeld, DanScavino, realDonaldTrump, jonathanadler, DamienHirst, HuffingtonPost, JoeNBC, tl1golf, SmaldinoLou, DebTate, Mikefa123, Jenn_Abrams, JaredWyand, moisemorancy, rjjulia, debra_tate9, VP, Shaddai289, iFunny, ABC, tyleroakley, KevinJacksonTBS, alfranken, megynkelly, newtgingrich]","[Colossis2, dllynnnnn, amyschumer, twitter, LauraJSilverman, FLOTUS, JaclynGlenn, JaclynGlenn, JaclynGlenn, GMA, YouTube, NRO, SarahKSilverman, Forbes, LeslieMarshall, JtMobleyFla, bawesome84, SouthwestAir, KABCRadio, GMA, Colossis2, dllynnnnn, NRO, KennedyNation, HillaryClinton, YouTube, SouthwestAir, FullFrontalSamB, JaclynGlenn, greggutfeld, DanScavino, realDonaldTrump, jonathanadler, DamienHirst, HuffingtonPost, LeslieMarshall, realDonaldTrump, JaclynGlenn, JoeNBC, tl1golf, SmaldinoLou, DebTate, Mikefa123, Jenn_Abrams, JaredWyand, JaclynGlenn, moisemorancy, rjjulia, JaclynGlenn, debra_tate9, Mikefa123, VP, Jenn_Abrams, JaredWyand, JaclynGlenn, Jenn_Abrams, JaredWyand, NRO, NRO, Shaddai289, dllynnnnn, iFunny, Colossis2, dllynnnnn, JaclynGlenn, JaredWyand, realDonaldTrump, Jenn_Abrams, JaredWyand, NRO, ABC, JaclynGlenn, JaclynGlenn, tyleroakley, ABC, Jenn_Abrams, JaredWyand, Jenn_Abrams, JaredWyand, KevinJacksonTBS, amyschumer, Jenn_Abrams, JaredWyand, alfranken, megynkelly, newtgingrich]"
0101limey0101,"[wikileaks, bearkilgore, hiphop247, BowieHutcheson, TCDwriter, hiphop247help, FoxNews, infowars, Project_Veritas, nia4_trump, bfraser747, JCArgentum, PodestaEmails18, Teri423, TrumpMartel, MichaelCohen212, realamerican, LenaMessick, medellinemil, Pamela_Moore13, newtgingrich, realDonaldTrump, kcphaeton, AussiesInTheUSA, dailyexpressuk, VictorB123, MattsonTina, mynameislauraca, seanhannity, PiratePigeon, OliMauritania, WDFx2EU7, Morning_Joe, JoeNBC, peterschweizer, rudygiulianiGOP, peterdukephoto, JaredWyand, AmericanSoWoke, bakedalaska, JoshLeCash, RealJamesWoods]","[wikileaks, bearkilgore, hiphop247, BowieHutcheson, TCDwriter, hiphop247help, FoxNews, infowars, Project_Veritas, nia4_trump, bfraser747, JCArgentum, PodestaEmails18, Teri423, TrumpMartel, MichaelCohen212, realamerican, LenaMessick, medellinemil, Pamela_Moore13, newtgingrich, realDonaldTrump, infowars, kcphaeton, AussiesInTheUSA, infowars, dailyexpressuk, VictorB123, MattsonTina, mynameislauraca, seanhannity, BowieHutcheson, PiratePigeon, OliMauritania, WDFx2EU7, Morning_Joe, JoeNBC, peterschweizer, rudygiulianiGOP, peterdukephoto, JaredWyand, realDonaldTrump, AmericanSoWoke, bakedalaska, JoshLeCash, RealJamesWoods]"
01801icecream,"[Cronikeys, ScottAdamsSays, gfallar, KimDotcom, thomasonphx, JudgeJeanine, DonaldJTrumpJr, WSJ, JakobWakuzi, geoffkeene, jasoninthehouse, christe4, Snowden, Jeff__Yang, realDonaldTrump, jamescoleman07, Hunte

In [17]:
users_active['handles_hashtags_stringified'] = users_active['handles_hashtags_all'].apply(lambda x: ' '.join(map(str, x)))

In [18]:
corpus = list(users_active['handles_hashtags_stringified'])

In [19]:
''' each string in the list contains all the hashtags/handles for one user 
    (contains duplicates if a hashtag/handle was used more than once) '''
corpus

['Mahasidi tim_oballa HassanAliJoho Amicos UhurutoCorruptionTrain Mahasidi AIRTEL_KE DeportBobCollymore HassanAliJoho Mahasidi viviankenya GainWithXtianDela Mahasidi GainWithVerifiedCofek shipetamathias GainWithVerifiedCofek UhurutoCorruptionTrain MaskiniMilionea AJENews IWantAPhoneThatCan UhurutoCorruptionTrain GainWithVerifiedCofek GainWithYahBoiSelfie IWantAPhoneThatCan njooro Mahasidi Fisi JailAwaitsYou GainWithVerifiedCofek paulkiarie001 ngathokaranja AJEnglish SonkoTheDrunk ngathokaranja AJEnglish SafaricomLtd DeportBobCollymore IWantAPhoneThatCan',
 'Colossis2 dllynnnnn amyschumer twitter LauraJSilverman FLOTUS JaclynGlenn JaclynGlenn JaclynGlenn GMA YouTube NRO SarahKSilverman Forbes LeslieMarshall JtMobleyFla bawesome84 SouthwestAir KABCRadio GMA Colossis2 dllynnnnn NRO KennedyNation HillaryClinton YouTube SouthwestAir FullFrontalSamB JaclynGlenn greggutfeld DanScavino realDonaldTrump jonathanadler DamienHirst HuffingtonPost LeslieMarshall realDonaldTrump JaclynGlenn JoeNBC tl

### hashtags/handles that have been used by at least 20 distinct users (min_df=20)

In [20]:
# constructs a dataframe with only the hashtags that appeared in 20 
vectorizer = sk_text.CountVectorizer(min_df=20)
matrix = vectorizer.fit_transform(corpus)
df = pd.DataFrame(matrix.toarray(), index=users_active.index, columns=vectorizer.get_feature_names())

In [21]:
# columns are hashtags and handles and values are frequency of handle/hashtag for a given user
df.head()

,_altright_anew,_carja,_cfj_,_makada_,_proud_american,_realvalentina_,a_miller48,abbydphillip,abbymartin,abbymartinm,...,zachhaller,zaibatsunews,zaidjilani,zekejmiller,zerohedge,zhaabowekwe,zigmanfreud,zika,zimmermanrob,zip90210
screen_name,,,,,,,,,,,,,,,,,,,,,
001comedian,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
007darB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0101limey0101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01801icecream,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0269Andy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df.shape

(17066, 4035)

In [25]:
# same matrix just swapped rows and columns
df.transpose()

screen_name,001comedian,007darB,0101limey0101,01801icecream,0269Andy,08awright,0UlJA,0ckam,0ectoplasm0,0hioBromo,...,zohrehnabahat,zombifiedmom,zomboid_muse,zoocoup,zorabet,zpr_media,zulahni,zvazda,zwraithz,zzubin
_altright_anew,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_carja,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_cfj_,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_makada_,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_proud_american,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
_realvalentina_,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
a_miller48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abbydphillip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abbymartin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abbymartinm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
